In [81]:
import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [82]:
sched = pd.read_excel('weekly_schedule.xlsx')

In [83]:
playoff_games_by_team_week = []

for team in sched.columns.to_list()[3:]:
    playoff_games_by_team_week.append([team] + sched.query("Week >= 19")[team].to_list())
    
playoff_games_by_team_week

playoff_games_weekly = pd.DataFrame(playoff_games_by_team_week, columns=['TEAM', 'Week1', 'Week2', 'Week3'])
playoff_games_weekly['Week1/2'] = playoff_games_weekly['Week1'] + playoff_games_weekly['Week2']
playoff_games_weekly['Week2/3'] = playoff_games_weekly['Week2'] + playoff_games_weekly['Week3']
playoff_games_weekly['Week1/3'] = playoff_games_weekly['Week1'] + playoff_games_weekly['Week3']

# Data from hashtagbasketball uses non-standard team abbreviations so they'll replaced later on.
team_name_map = {'BRO': 'BKN',
                 'PHX': 'PHO',
                 'NOP': 'NOR',
                 'OKL': 'OKC'}

In [84]:
sched.iloc[:, :3].head() # All teams share these same 3 columns

,Week Type,Week,Date
0,regular,1,2020-12-21
1,regular,2,2020-12-28
2,regular,3,2021-01-04
3,regular,4,2021-01-11
4,regular,5,2021-01-18


In [85]:
"""Extract these common columns. Will be used tranpose the data."""
base_sched = sched.iloc[:, :3]

In [86]:
schedule_by_team = []

def process_schedule(df, team):
    # Horizontally join the base schedule with the team's number of games
    team_sched = pd.concat([base_sched, df], axis=1)
    # Rename columns
    team_sched.columns = ['Week_Type', 'Week', 'Date', 'Number_of_Games']
    # Add new team column
    team_sched['Team'] = team
    # Append to list used to concat all teams schedules into one dataframe
    schedule_by_team.append(team_sched)

[process_schedule(sched[team], team) for team in sched.columns.to_list()[3:]]

len(schedule_by_team) == 30 # Should be 30 entries

True

In [87]:
"""Vertically append all teams schedules into one dataframe. Now
the data looks more relational instead of like a pivot table.
"""
final_schedule = pd.concat(schedule_by_team)

In [88]:
final_schedule.to_csv('weekly_schedule_transposed.csv', index=False)

In [89]:
max_games = max(final_schedule['Number_of_Games'])
final_schedule.query('Number_of_Games == @max_games')

,Week_Type,Week,Date,Number_of_Games,Team
12,regular,13,2021-03-15,5,CHI
3,regular,4,2021-01-11,5,IND
3,regular,4,2021-01-11,5,PHI
12,regular,13,2021-03-15,5,PHO
18,playoffs,19,2021-04-26,5,POR
9,regular,10,2021-02-22,5,WAS
12,regular,13,2021-03-15,5,WAS


In [90]:
max_playoff_games = max(final_schedule.query("Week_Type == 'playoffs'")['Number_of_Games'])
final_schedule.query("Number_of_Games == @max_playoff_games and Week_Type == 'playoffs'")

,Week_Type,Week,Date,Number_of_Games,Team
18,playoffs,19,2021-04-26,5,POR


In [106]:
column_renames = {'Number_of_Games': 'Total_Playoff_Games', 'Team': 'TEAM'}

total_playoff_games_by_team = final_schedule[['Team', 'Week', 'Number_of_Games']].query("Week >= 19")\
.groupby(by=['Team']).sum().reset_index().rename(columns=column_renames)

total_playoff_games_by_team[['TEAM', 'Total_Playoff_Games']]\
.reset_index()\
.set_index('TEAM')\
.join(playoff_games_weekly.set_index('TEAM'))\
.reset_index()\
.drop(columns=['index'])\
.sort_values(by=['Total_Playoff_Games', 'Week1', 'Week2', 'Week3'], ascending=False)\
.style.background_gradient(cmap='RdYlGn')\

,TEAM,Total_Playoff_Games,Week1,Week2,Week3,Week1/2,Week2/3,Week1/3
24,POR,12,5,3,4,8,7,9
10,HOU,12,4,4,4,8,8,8
27,TOR,12,4,4,4,8,8,8
7,DEN,11,4,4,3,8,7,7
9,GSW,11,4,4,3,8,7,7
16,MIL,11,4,4,3,8,7,7
17,MIN,11,4,4,3,8,7,7
4,CHI,11,4,3,4,7,7,8
13,LAL,11,4,3,4,7,7,8
14,MEM,11,4,3,4,7,7,8


In [92]:
# Scrape the player projections from hashtagbasketball then link it to schedule data
from selenium import webdriver

driver = webdriver.Chrome('../selenium-resources/chromedriver.exe')

In [93]:
driver.get('https://hashtagbasketball.com/fantasy-basketball-points-league-rankings')

In [94]:
driver.close()

In [95]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

site= "https://hashtagbasketball.com/fantasy-basketball-points-league-rankings"
hdr = {'User-Agent': 'Mozilla/5.0'}
req = Request(site,headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page)

t = soup.find(id = 'ContentPlaceHolder1_GridView1')

In [96]:
players_df = pd.read_html(t.decode())

In [97]:
players_df[0].to_csv('player_projections_20_21.csv', index=False)

In [98]:
cols = {'R#': 'Per Game Rank', 
        'NAME': 'Player', 
        'TOTAL': 'Per Game Avg', 
        'Total_Playoff_Games': 'Total Playoff Games',
        'POS': 'Position'}

def fix_team_name(team):
    
    if team in team_name_map.keys():
        return team_name_map[team]
    
    return team

# Standardize team names
players_df[0]['TEAM'] = players_df[0]['TEAM'].apply(fix_team_name)

players_projection_games = players_df[0].set_index('TEAM')\
.join(total_playoff_games_by_team.set_index('TEAM'))\
.reset_index()\
.rename(columns=cols)\
.drop(columns=['Week'])\
.query("TEAM != 'TEAM'")

players_projection_games.to_csv('players_projection_games.csv', index=False)
players_projection_games.query("TEAM == 'ATL'").head()

,TEAM,Per Game Rank,Player,Per Game Avg,Position,GP,PTS,TREB,AST,STL,BLK,TO,Total Playoff Games
0,ATL,12,Trae Young,42.85,PG,68,26.80,4.20,14.40,3.00,0.30,-5.85,11.0
1,ATL,44,John Collins,33.88,"PF,C",65,18.70,10.08,2.25,2.40,3.30,-2.85,11.0
2,ATL,60,Clint Capela,31.64,C,59,11.90,12.84,2.10,2.40,4.80,-2.40,11.0
3,ATL,123,Bogdan Bogdanovic,24.81,"SG,SF,PF",62,14.40,3.96,4.95,3.30,0.60,-2.40,11.0
4,ATL,161,Danilo Gallinari,21.65,"SF,PF",58,14.30,4.80,2.25,1.80,0.30,-1.80,11.0


In [101]:
convert_to_num_cols = ['Per Game Avg', 'GP', 'PTS', 'TREB', 'AST', 'STL', 'BLK', 'TO']
column_order = ['TEAM', 'Player', 'Position', 'Per Game Rank']\
               + convert_to_num_cols + ['Total Playoff Games', 'Week1', 'Week2', 'Week3', 'Week1/2', 'Week2/3', 'Week1/3']
players_projection_games[convert_to_num_cols] = players_projection_games[convert_to_num_cols].apply(pd.to_numeric)

from itables import show

final = players_projection_games.set_index('TEAM')\
.join(playoff_games_weekly.set_index('TEAM'))\

show(final\
.reset_index()\
[column_order]\
#.sort_values(by=['Per Game Avg'], ascending=False)\
#.style.background_gradient(cmap='RdYlGn')
)

,TEAM,Player,Position,Per Game Rank,Per Game Avg,GP,PTS,TREB,AST,STL,BLK,TO,Total Playoff Games,Week1,Week2,Week3,Week1/2,Week2/3,Week1/3


In [109]:
sorting_criteria = {'tpg_pga_aa': {}}

final\
.reset_index()\
[column_order]\
.sort_values(by=['Total Playoff Games', 'Per Game Avg'], ascending=[False, False])\
.style.background_gradient(cmap='RdYlGn')